In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from tqdm import tqdm
import sys
sys.path.append('..')
import bets
nba = bets.nba()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
# We will want to see how our predicitons did vs the books
mod = pd.read_pickle('data/model/thrModv201.pkl')

In [91]:
bdf = pd.read_csv('data/csv/24-25Bets.csv')

In [96]:
moddf = bdf[bdf.tags.fillna('None').str.contains('ModPlay')].reset_index(drop=True)

In [97]:
moddf.profit.sum()

4864.14

In [98]:
moddf['date'] = [pd.to_datetime(x[:10]).strftime('%Y-%m-%d') for x in moddf.time_placed]

In [104]:
moddf.ev.describe(np.arange(.1,1,.1))

count    1306.000000
mean        0.012120
std         0.134905
min        -0.348900
10%        -0.124300
20%        -0.081300
30%        -0.055500
40%        -0.031000
50%        -0.005000
60%         0.020400
70%         0.052000
80%         0.098400
90%         0.174950
max         0.960100
Name: ev, dtype: float64

In [111]:
moddf.bet_info[1]

'Julius Randle: 3+ Threes Made Threes Made UTA Jazz @ MIN Timberwolves | Jamal Murray: 4+ Threes Made Threes Made DEN Nuggets @ HOU Rockets'

In [113]:
moddf.type

0         parlay
1         parlay
2         parlay
3       straight
4       straight
          ...   
1817      parlay
1818    straight
1819    straight
1820    straight
1821    straight
Name: type, Length: 1822, dtype: object

In [114]:
moddf[moddf.type=='straight']

,bet_id,sportsbook,type,status,odds,closing_line,ev,amount,profit,time_placed,time_settled,bet_info,tags,sports,leagues,date
3,PA6345029851W,Bet365,straight,SETTLED_LOSS,2.45,2.67,-0.0824,8.50,-8.5,04/13/2025 10:51:00 GMT,04/13/2025 20:40:00 GMT,Jamal Murray: 3+ Threes Made Threes Made DEN N...,ModPlay,Basketball,NBA,2025-04-13
4,PA6345029852W,Bet365,straight,SETTLED_LOSS,1.87,2.05,-0.0880,8.00,-8.0,04/13/2025 10:51:00 GMT,04/13/2025 20:40:00 GMT,Julius Randle: 2+ Threes Made Threes Made UTA ...,ModPlay,Basketball,NBA,2025-04-13
5,SA9933457251W,Bet365,straight,SETTLED_LOSS,3.60,3.65,-0.0128,5.50,-5.5,04/13/2025 10:52:00 GMT,04/13/2025 20:40:00 GMT,Julius Randle: 3+ Threes Made Threes Made UTA ...,ModPlay,Basketball,NBA,2025-04-13
6,SA9933457252W,Bet365,straight,SETTLED_LOSS,4.50,4.69,-0.0396,8.00,-8.0,04/13/2025 10:52:00 GMT,04/13/2025 20:40:00 GMT,Jamal Murray: 4+ Threes Made Threes Made DEN N...,ModPlay,Basketball,NBA,2025-04-13
7,O/0586450/0003026,Fanduel Sportsbook,straight,SETTLED_VOID,2.72,NaN,NaN,7.25,0.0,04/13/2025 10:56:23 GMT,04/13/2025 18:58:17 GMT,Ja Morant 3+ Made Threes Dallas Mavericks @ Me...,ModPlay,Basketball,NBA,2025-04-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815,638651517117652754,Draftkings Sportsbook,straight,SETTLED_WIN,4.90,NaN,NaN,5.00,19.5,10/21/2024 23:55:12 GMT,10/23/2024 04:41:55 GMT,3+ Nickeil Alexander-Walker Three Pointers Mad...,ModPlay,Basketball,NBA,2024-10-21
1818,638651517132920291,Draftkings Sportsbook,straight,SETTLED_LOSS,1.91,1.96,-0.0267,10.00,-10.0,10/21/2024 23:55:12 GMT,10/23/2024 01:50:00 GMT,2+ Payton Pritchard Three Pointers Made NY Kni...,ModPlay,Basketball,NBA,2024-10-21
1819,638651517117673978,Draftkings Sportsbook,straight,SETTLED_LOSS,3.65,NaN,NaN,5.00,-5.0,10/21/2024 23:55:12 GMT,10/23/2024 01:50:00 GMT,3+ Payton Pritchard Three Pointers Made NY Kni...,ModPlay,Basketball,NBA,2024-10-21
1820,638651517117482436,Draftkings Sportsbook,straight,SETTLED_LOSS,7.00,8.44,-0.1709,5.00,-5.0,10/21/2024 23:55:12 GMT,10/23/2024 01:47:49 GMT,3+ Josh Hart Three Pointers Made NY Knicks @ B...,ModPlay,Basketball,NBA,2024-10-21


In [118]:

moddf[['ev','odds','profit']].corr()

,ev,odds,profit
ev,1.000000,0.083807,0.052417
odds,0.083807,1.000000,0.003356
profit,0.052417,0.003356,1.000000


In [122]:

moddf[moddf.type=='straight'][['ev','odds','profit']].corr()

,ev,odds,profit
ev,1.000000,0.365474,0.152994
odds,0.365474,1.000000,0.054973
profit,0.152994,0.054973,1.000000


In [131]:
##split out bets to be more readable
len([x for x in moddf.bet_info if x.find(':')==-1])

1012

In [135]:
moddf[moddf.bet_info.str.contains(':')].sportsbook.value_counts()

sportsbook
Bet365    785
BetMGM     25
Name: count, dtype: int64

In [139]:

moddf[(~moddf.bet_info.str.contains(':')) & (moddf.sportsbook.str.startswith('B'))]

,bet_id,sportsbook,type,status,odds,closing_line,ev,amount,profit,time_placed,time_settled,bet_info,tags,sports,leagues,date
27,201BS8T042,BetMGM,parlay,SETTLED_LOSS,21.87,17.99,0.2154,3.00,-3.00,04/10/2025 11:24:52 GMT,04/11/2025 04:40:00 GMT,Yes Tobias Harris to make 2+ three-pointers Ne...,ModPlay,Basketball,NBA,2025-04-10
28,201BS8T03F,BetMGM,straight,SETTLED_LOSS,6.25,6.12,0.0204,3.00,-3.00,04/10/2025 11:24:52 GMT,04/11/2025 04:40:00 GMT,Yes Scotty Pippen to make 3+ three-pointers Mi...,ModPlay,Basketball,NBA,2025-04-10
35,201BS8SZZ6,BetMGM,straight,SETTLED_WIN,3.50,2.94,0.1914,3.00,7.50,04/10/2025 11:24:52 GMT,04/11/2025 01:09:58 GMT,Yes Tobias Harris to make 2+ three-pointers Ne...,ModPlay,Basketball,NBA,2025-04-10
39,201B08Y6D2,BetMGM,straight,SETTLED_LOSS,3.10,3.09,0.0023,10.00,-10.00,04/09/2025 22:04:27 GMT,04/10/2025 05:10:00 GMT,Yes Keldon Johnson to make 2+ three-pointers S...,ModPlay,Basketball,NBA,2025-04-09
40,201B08Y6CE,BetMGM,straight,SETTLED_LOSS,8.00,NaN,NaN,6.50,-6.50,04/09/2025 22:04:27 GMT,04/10/2025 05:10:00 GMT,Yes Keldon Johnson to make 3+ three-pointers S...,ModPlay,Basketball,NBA,2025-04-09
68,BX1831210141W,Bet365,straight,SETTLED_WIN,2.60,2.59,0.0039,9.00,14.40,04/08/2025 16:33:00 GMT,04/09/2025 00:30:00 GMT,Jalen Brunson - Over 2.5 Threes Made O/U BOS C...,ModPlay,Basketball,NBA,2025-04-08
157,CX8783963002W,Bet365,straight,SETTLED_LOSS,2.20,2.27,-0.0308,10.00,-10.00,03/27/2025 16:55:00 GMT,03/28/2025 00:10:00 GMT,Jaren Jackson Jr - Under 1.5 Threes Made O/U M...,ModPlay,Basketball,NBA,2025-03-27
163,200P1FJWJ7,BetMGM,straight,SETTLED_LOSS,3.40,3.31,0.0276,8.40,-8.40,03/26/2025 18:23:52 GMT,03/27/2025 02:02:47 GMT,Yes Scottie Barnes to make 2+ three-pointers T...,ModPlay,Basketball,NBA,2025-03-26
164,200P1FJWPP,BetMGM,straight,SETTLED_WIN,2.65,2.59,0.0232,8.80,14.52,03/26/2025 18:23:52 GMT,03/27/2025 02:02:47 GMT,Yes Rui Hachimura to make 2+ three-pointers Lo...,ModPlay,Basketball,NBA,2025-03-26
165,200P1FJWJU,BetMGM,parlay,SETTLED_LOSS,9.01,8.57,0.0514,5.00,-5.00,03/26/2025 18:23:52 GMT,03/27/2025 02:02:47 GMT,Yes Scottie Barnes to make 2+ three-pointers T...,ModPlay,Basketball,NBA,2025-03-26


In [142]:
moddf[moddf.sportsbook.isin(['DraftKings Sportsbook','FanDuel Sportsbook'])]

,bet_id,sportsbook,type,status,odds,closing_line,ev,amount,profit,time_placed,time_settled,bet_info,tags,sports,leagues,date


In [149]:
moddf[(moddf.bet_info.str.contains('Under')) & (moddf.sportsbook=='Draftkings Sportsbook')].loc[1494].bet_info

'2+ Keldon Johnson Three Pointers Made SA Spurs @ SAC Kings | Under 1.5 Rui Hachimura Three Pointers Made O/U LA Lakers @ UTA Jazz | Under 3.5 Dalton Knecht Three Pointers Made O/U LA Lakers @ UTA Jazz'

In [ ]:
moddf.

In [ ]:
def convertPikkit(df,value):
    '''
    Take Pikkit download extract: player name, game date, value, book, odds and wining to format to be saved in db.
    Inputs: df of the pikkit data, value - this is the market (ie Threes, Points) to save down
    Output: Dataframe that will replicate what is now stored in the db.
    '''



In [151]:
odds = pd.read_csv('

type
parlay           3508.75
round_robin_2     293.22
round_robin_3     -13.25
round_robin_4      -0.50
straight         1075.92
Name: profit, dtype: float64

In [35]:
import os
import glob
import datetime as dt

In [36]:
[dir for dir in os.listdir(os.getcwd() + '/data/nbaData')]

['2023-24gameLines.pkl',
 '2024-25gameLines.pkl',
 '2023-24rebsAsstQ3.pkl',
 '2024-25firstBaskets.pkl',
 '2024-25rebsAsstQ3.pkl',
 '2024-25pointsThreespm.pkl',
 '2023-24q1.pkl',
 '2024-25pointsThrees.pkl',
 '2024-25q1.pkl',
 '2024-25firstBasketspm.pkl']

In [39]:
pd.read_pickle(files[1])

{'0_0': {'id': 'a7d6bf35d58918099f6a139a4e67908d',
  'sport_key': 'basketball_nba',
  'sport_title': 'NBA',
  'commence_time': '2025-03-27T23:00:00Z',
  'home_team': 'Cleveland Cavaliers',
  'away_team': 'San Antonio Spurs',
  'bookmakers': [{'key': 'draftkings',
    'title': 'DraftKings',
    'last_update': '2025-03-27T21:55:08Z',
    'markets': [{'key': 'h2h',
      'last_update': '2025-03-27T21:55:08Z',
      'outcomes': [{'name': 'Cleveland Cavaliers', 'price': -950},
       {'name': 'San Antonio Spurs', 'price': 625}]},
     {'key': 'spreads',
      'last_update': '2025-03-27T21:55:08Z',
      'outcomes': [{'name': 'Cleveland Cavaliers',
        'price': -108,
        'point': -14.0},
       {'name': 'San Antonio Spurs', 'price': -112, 'point': 14.0}]},
     {'key': 'totals',
      'last_update': '2025-03-27T21:55:08Z',
      'outcomes': [{'name': 'Over', 'price': -110, 'point': 236.5},
       {'name': 'Under', 'price': -110, 'point': 236.5}]}]},
   {'key': 'fanduel',
    'title':

In [37]:
files = glob.glob('data/nbaData/*.pkl')

'data/nbaData/2024-25firstBaskets.pkl'

In [154]:
thrs = pd.read_pickle('data/nbaData/2024-25pointsThrees.pkl')
fb = pd.read_pickle('data/nbaData/2024-25firstBasketspm.pkl')


In [155]:
test = list(fb.items())[:100]

In [377]:
import json

In [161]:
test

[('0_0',
  {'id': 'a7d6bf35d58918099f6a139a4e67908d',
   'sport_key': 'basketball_nba',
   'sport_title': 'NBA',
   'commence_time': '2025-03-27T23:00:00Z',
   'home_team': 'Cleveland Cavaliers',
   'away_team': 'San Antonio Spurs',
   'bookmakers': [{'key': 'draftkings',
     'title': 'DraftKings',
     'last_update': '2025-03-27T21:55:08Z',
     'markets': [{'key': 'player_first_basket',
       'last_update': '2025-03-27T21:55:59Z',
       'outcomes': [{'name': 'Yes',
         'description': 'Jarrett Allen',
         'price': 550},
        {'name': 'Yes', 'description': 'Chris Paul', 'price': 1400},
        {'name': 'Yes', 'description': 'Devin Vassell', 'price': 700},
        {'name': 'Yes', 'description': 'Dean Wade', 'price': 1500},
        {'name': 'Yes', 'description': 'Harrison Barnes', 'price': 1000},
        {'name': 'Yes', 'description': 'Donovan Mitchell', 'price': 400},
        {'name': 'Yes', 'description': 'Bismack Biyombo', 'price': 1200},
        {'name': 'Yes', 'descr

In [202]:
df = pd.DataFrame()
missing = []
idx = ['description','date','name','mrkt',]
#first basket needs a different idx because it does not have point
mkts = ['threes','points','player_first_basket','player_first_team_basket','spread']
for t in tqdm(list(fb.items())):
    try:
        home = t[1].get('home_team')
        away = t[1].get('away_team')
        date = (pd.to_datetime(t[1].get('commence_time')) - dt.timedelta(hours = 4)).strftime('%Y-%m-%d')
        for books in t[1].get('bookmakers'):
            bk = books.get('key')
            for mrkts in books.get('markets'):
                d = mrkts.get('outcomes')
                m = mrkts.get('key')
                temp = pd.DataFrame(d)
                alt = 'alt' if m.find('alt')>-1 else 'OU'
                temp['date'] = date
                temp = temp.rename(columns = {'price':'{}_{}_price'.format(bk,alt)} if m.find('basket')==-1 else {'price':'{}'.format(bk)})
                temp['mrkt'] = [l for l in m.split('_') if l in mkts][0] if m.find('basket')==-1 else m
                df = pd.concat([df,temp.groupby(idx).max()])
    except:
      missing.append(t)  


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1805/1805 [00:07<00:00, 256.83it/s]


In [204]:
df

,,,,draftkings,fanduel
description,date,name,mrkt,,
Bismack Biyombo,2025-03-27,Yes,player_first_basket,1200.0,NaN
Chris Paul,2025-03-27,Yes,player_first_basket,1400.0,NaN
Darius Garland,2025-03-27,Yes,player_first_basket,500.0,NaN
Dean Wade,2025-03-27,Yes,player_first_basket,1500.0,NaN
Devin Vassell,2025-03-27,Yes,player_first_basket,700.0,NaN
...,...,...,...,...,...
Myles Turner,2025-01-25,Yes,player_first_basket,NaN,900.0
Pascal Siakam,2025-01-25,Yes,player_first_basket,NaN,700.0
Stephon Castle,2025-01-25,Yes,player_first_basket,NaN,650.0


In [205]:
names = ['fanduel','draftkings','betmgm','espnbet']
mkts = ['threes','points','player_first_basket','player_first_team_basket','spread']
for book in names:
    try:
        df[book] = df['{}_OU_price'.format(book)].fillna(df['{}_alt_price'.format(book)])
    except:
        print('No {} odds'.format(book))
df = df.groupby(idx).min().filter(names).reset_index()
for m in mkts:
    temp = df[df.mrkt.str.contains(m)].drop(['mrkt'],axis=1)

    

No fanduel odds
No draftkings odds
No betmgm odds
No espnbet odds


In [206]:
df

,description,date,name,mrkt,fanduel,draftkings
0,A.J. Green,2024-11-13,Yes,player_first_basket,1000.0,NaN
1,A.J. Green,2024-12-21,Yes,player_first_basket,900.0,750.0
2,A.J. Lawson,2025-03-08,Yes,player_first_basket,NaN,800.0
3,A.J. Lawson,2025-03-08,Yes,player_first_team_basket,NaN,600.0
4,A.J. Lawson,2025-03-10,Yes,player_first_basket,NaN,950.0
...,...,...,...,...,...,...
12696,Zion Williamson,2025-03-13,Yes,player_first_team_basket,220.0,NaN
12697,Zion Williamson,2025-03-17,Yes,player_first_basket,600.0,500.0
12698,Zion Williamson,2025-03-17,Yes,player_first_team_basket,210.0,225.0
12699,Zion Williamson,2025-03-19,Yes,player_first_basket,700.0,400.0


In [207]:
temp = df.reset_index()[df.reset_index().mrkt.str.contains('basket')].drop(['mrkt'],axis=1)
cols = [r[1] for r in nba.cur.execute('PRAGMA table_info(oddsFirstBasket)').fetchall()]
d = zip(temp.columns,cols)
temp = temp.rename(columns = dc)
for c in cols:
    if c not in temp.columns:
        temp[c] = None

In [210]:
nba.insert_data(temp.filter(cols),'oddsFirstBasket')

oddsFirstBasket has been updated with 12,701 rows


In [487]:
fb = pd.read_pickle('data/nbaData/2024-25firstBasketspm.pkl')

In [483]:
pd.read_sql('select * from oddsThrees',nba.conn).columns

Index(['name', 'game_date', 'side', 'amount', 'FDOdds', 'DKOdds', 'ESPNOdds',
       'MGMOdds', 'ModelOdds', 'betPlaced', 'betAmount', 'betBook'],
      dtype='object')

In [394]:
dk = df.loc['Devin Vassell'].draftkings_OU_price.fillna(df.loc['Devin Vassell'].draftkings_alt_price).to_frame('draftkings').dropna()
fd = df.loc['Devin Vassell'].fanduel_OU_price.fillna(df.loc['Devin Vassell'].fanduel_alt_price).to_frame('fanduel').dropna()

In [367]:
df.loc['Zion Williamson'].loc['2025-03-24'].head(50)

draftkings_alt_price  fanduel_alt_price  \
name  point mrkt                                                               
Over  14.5  player_points_alternate               -1800.0            -1600.0   
      17.5  player_points_alternate                -700.0                0.0   
      19.5  player_points_alternate                -425.0             -350.0   
      24.5  player_points_alternate                -135.0             -122.0   
      25.5  player_points_alternate                -115.0                0.0   
      29.5  player_points_alternate                 180.0              220.0   
      34.5  player_points_alternate                 450.0              500.0   
      39.5  player_points_alternate                1200.0             1060.0   
      44.5  player_points_alternate                3000.0             2000.0   
      25.5  player_points                             NaN                NaN   
Under 25.5  player_points                             NaN                NaN   
Over  14.5  player_points_alternate               -1800.0            -1600.0   
      17.5  player_points_alternate                -700.0                0.0   
      19.5  player_points_alternate                -425.0             -350.0   
      24.5  player_points_alternate                -135.0             -122.0   
      25.5  player_points_alternate                -115.0                0.0   
      29.5  player_points_alternate                 180.0              220.0   
      34.5  player_points_alternate                 450.0              500.0   
      39.5  player_points_alternate                1200.0             1060.0   
      44.5  player_points_alternate                3000.0             2000.0   
      25.5  player_points                             NaN                NaN   
Under 25.5  player_points                             NaN                NaN   
Over  14.5  player_points_alternate               -1800.0            -1600.0   
      17.5  player_points_alternate                -700.0                0.0   
      19.5  player_points_alternate                -425.0             -350.0   
      24.5  player_points_alternate                -135.0             -122.0   
      25.5  player_points_alternate                -115.0                0.0   
      29.5  player_points_alternate                 180.0              220.0   
      34.5  player_points_alternate                 450.0              500.0   
      39.5  player_points_alternate                1200.0             1060.0   
      44.5  player_points_alternate                3000.0             2000.0   
      25.5  player_points                             NaN                NaN   
Under 25.5  player_points                             NaN                NaN   
Over  14.5  player_points_alternate               -1800.0            -1600.0   
      17.5  player_points_alternate                -700.0                0.0   
      19.5  player_points_alternate                -425.0             -350.0   
      24.5  player_points_alternate                -135.0             -122.0   
      25.5  player_points_alternate                -115.0                0.0   
      29.5  player_points_alternate                 180.0              220.0   
      34.5  player_points_alternate                 450.0              500.0   
      39.5  player_points_alternate                1200.0             1060.0   
      44.5  player_points_alternate                3000.0             2000.0   
      25.5  player_points                             NaN                NaN   
Under 25.5  player_points                             NaN                NaN   
Over  14.5  player_points_alternate               -1800.0            -1600.0   
      17.5  player_points_alternate                -700.0                0.0   
      19.5  player_points_alternate                -425.0             -350.0   
      24.5  player_points_alternate                -135.0             -122.0   
      25.5  player_points_alternate  

In [13]:
nba.showTables

,type,name,tbl_name,rootpage,sql
0,table,rosters,rosters,4,"CREATE TABLE rosters(\n teamId TEXT,\n s..."
1,table,players,players,2,"CREATE TABLE players(\n player_id TEXT,..."
2,table,teams,teams,6,"CREATE TABLE teams(\n team_id TEXT,\n te..."
3,table,plyrLogs,plyrLogs,123,"CREATE TABLE plyrLogs (player_id TEXT,\nteam_i..."
4,view,team_game,team_game,0,"CREATE VIEW team_game as \nSELECT season, game..."
5,table,shotsAllowed,shotsAllowed,582,"CREATE TABLE shotsAllowed (team_id TEXT,\ngame..."
6,view,team_def,team_def,0,"CREATE VIEW team_def AS\nselect \nseason, subs..."
7,table,teamLog,teamLog,7,"CREATE TABLE teamLog(\n game_id TEXT,\n..."
8,table,preds,preds,11580,CREATE TABLE preds\n (game_...
9,view,pgames,pgames,0,CREATE VIEW pgames \n AS\nSELECT \n--identi...


In [21]:
pd.read_sql('select * from oddsThrees',nba.conn).columns

Index(['name', 'game_date', 'side', 'amount', 'FDOdds', 'DKOdds', 'ESPNOdds',
       'MGMOdds', 'ModelOdds', 'betPlaced', 'betAmount', 'betBook'],
      dtype='object')

In [20]:
df.rename({'description':'name','date':'game_date','name':'side','point':'amount','fanduel':'FDOdds','draftkings':'DKOdds',

,description,date,name,mrkt,point,draftkings_OU_price,draftkings_alt_price,fanduel_OU_price,fanduel_alt_price,fanduel,draftkings
0,Chris Paul,2025-03-27,Over,player_points,6.5,-105.0,NaN,NaN,NaN,NaN,-105.0
1,Chris Paul,2025-03-27,Under,player_points,6.5,-125.0,NaN,NaN,NaN,NaN,-125.0
2,Darius Garland,2025-03-27,Over,player_points,20.5,-110.0,NaN,NaN,NaN,NaN,-110.0
3,Darius Garland,2025-03-27,Under,player_points,20.5,-120.0,NaN,NaN,NaN,NaN,-120.0
4,Devin Vassell,2025-03-27,Over,player_points,19.5,-110.0,NaN,NaN,NaN,NaN,-110.0
...,...,...,...,...,...,...,...,...,...,...,...
555266,Victor Wembanyama,2025-01-25,Over,player_threes_alternate,3.5,NaN,NaN,NaN,105.0,105.0,NaN
555267,Victor Wembanyama,2025-01-25,Over,player_threes_alternate,4.5,NaN,NaN,NaN,220.0,220.0,NaN
555268,Victor Wembanyama,2025-01-25,Over,player_threes_alternate,5.5,NaN,NaN,NaN,460.0,460.0,NaN
555269,Victor Wembanyama,2025-01-25,Over,player_threes_alternate,6.5,NaN,NaN,NaN,800.0,800.0,NaN


In [7]:
games = nba.get_games('2022-10-01','2025-04-20')

In [4]:
cols = [r[1] for r in nba.cur.execute('PRAGMA table_info(plyrLogs)').fetchall()]

In [8]:
table = open('data/sql/playerQ1.sql','r').read()

In [15]:
games[:1]

,GAME_DATE,TEAM_ID,GAME_ID
0,2025-04-13,1610612741,0022401191


In [16]:
nba.get_logs(games[:1].GAME_DATE.values)

started player logs at 11:28


,PLAYER_ID,TEAM_ID,GAME_ID,GAME_DATE,MIN,FTM,FTA,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,DD2,TD3
0,201935,1610612746,0022401198,2025-04-13,48.400000,8,9,7,10,6,2,2,0,3,8,39,5,1,0
1,1628420,1610612756,0022401200,2025-04-13,21.033333,0,0,3,2,0,0,0,1,2,0,8,-14,0,0
2,1630639,1610612761,0022401197,2025-04-13,21.300000,7,7,0,0,1,0,0,0,1,3,14,-5,0,0
3,1631111,1610612766,0022401187,2025-04-13,14.788333,0,0,3,2,0,0,0,0,1,0,8,2,0,0
4,1631099,1610612758,0022401200,2025-04-13,22.716667,0,0,7,2,0,0,0,0,0,0,6,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1642358,1610612764,0022401190,2025-04-13,48.000000,0,0,1,8,2,1,0,1,2,0,14,1,0,0
295,1642271,1610612762,0022401195,2025-04-13,28.516667,1,1,3,3,0,0,2,3,6,2,15,-25,0,0
296,1627739,1610612746,0022401198,2025-04-13,35.365000,0,0,1,6,1,3,0,0,4,1,11,11,0,0
297,1628989,1610612741,0022401191,2025-04-13,16.266667,1,2,2,2,0,0,0,0,2,1,18,4,0,0


In [ ]:
n

In [30]:
nba.update_player_log(games[:1].GAME_DATE.values,qtr=1)

started player logs at 10:23
getting first buckets : at 10:23


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:43<00:00,  2.88s/it]


	completed at 10:24
started rebounds at 10:24
ended rebounds at 10:24
starting advanced box at 10:24


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:12<00:00,  4.82s/it]


completed adv box at 10:25
start player shots at 10:25


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.22s/it]


completed player shots at 10:25
plyrLogs has been updated with 257 rows


,player_id,team_id,game_id,game_date,min,ftm,fta,reb,ast,tov,...,paint_fgm,mid_fgm,mid_fga,lc_fgm,lc_fga,rc_fgm,rc_fga,abv_fgm,abv_fga,Starter
0,1628398,1610612749,0022401192,2025-04-13,12.000000,1,1,2,2,1,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,5.0,1
1,1630526,1610612740,0022401196,2025-04-13,5.450000,1,2,3,0,1,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0
2,1631221,1610612756,0022401200,2025-04-13,6.433333,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0
3,203901,1610612740,0022401196,2025-04-13,4.766667,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0
4,1642278,1610612752,0022401188,2025-04-13,2.966667,0,0,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,1629022,1610612755,0022401191,2025-04-13,10.166667,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0,0
253,1630577,1610612759,0022401197,2025-04-13,8.533333,0,0,2,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0
254,1642259,1610612764,0022401190,2025-04-13,5.916667,3,4,2,1,0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0
255,1631123,1610612749,0022401192,2025-04-13,8.350000,0,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [23]:
from nba_api.stats.endpoints import BoxScoreAdvancedV3

In [31]:
import time
gs = games[games.GAME_DATE<='2024-03-21'].GAME_DATE.unique()

In [29]:
for i,g in enumerate(gs):
    df = nba.update_player_log([g],qtr=1)
    nba.insert_data(df,'plyrQ1Logs')
    if i % np.random.randint(1,3) == 0:
        time.sleep(np.random.randint(0,13))

started player logs at 10:51
getting first buckets : at 10:51


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:27<00:00,  3.94s/it]


	completed at 10:52
started rebounds at 10:52
ended rebounds at 10:52
starting advanced box at 10:52


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:32<00:00,  4.61s/it]


completed adv box at 10:52
start player shots at 10:52


...

	completed at 17:16
started rebounds at 17:16
ended rebounds at 17:16
starting advanced box at 17:16


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.19s/it]


completed adv box at 17:16
start player shots at 17:16


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.84s/it]
/Users/ericcoxon/code/bets/nba/../bets.py:599: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  logrbs = log.merge(rbs,how='left',on=['PLAYER_ID','GAME_ID','TEAM_ID','GAME_DATE']).fillna(0)


completed player shots at 17:17
plyrLogs has been updated with 38 rows
plyrQ1Logs has been updated with 38 rows


In [9]:
for g in games.GAME_DATE.unique()[:5]:
    df = nba.update_player_log([g],qtr=1)


started player logs at 14:55
getting first buckets : at 14:55


  0%|                                                                                                                                                          | 0/15 [00:00<?, ?it/s]Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:44<00:00,  2.98s/it]


	completed at 14:56
started rebounds at 14:56
ended rebounds at 14:56
starting advanced box at 14:56


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:12<00:00,  4.84s/it]


completed adv box at 14:57
start player shots at 14:57


...

completed player shots at 15:00
plyrLogs has been updated with 257 rows
started player logs at 15:00
getting first buckets : at 15:00


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:15<00:00,  3.08s/it]


	completed at 15:00
started rebounds at 15:00
ended rebounds at 15:00
starting advanced box at 15:00


 20%|█████████████████████████████▍                                                                                                                     | 1/5 [00:33<02:12, 33.22s/it]


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)